In [ ]:
(in-package "ACL2")

In [ ]:
; Basic list predicates
(defun nat-listp (l)
  (if (consp l)
      (and (natp (car l))
           (nat-listp (cdr l)))
    (null l)))

In [ ]:
; fold-left: process list from left to right
; (fold-left f init '(a b c)) = (f (f (f init a) b) c)
(defun fold-left (f acc l)
  (if (consp l)
      (fold-left f (funcall f acc (car l)) (cdr l))
    acc))

In [ ]:
; fold-right: process list from right to left
; (fold-right f init '(a b c)) = (f a (f b (f c init)))
(defun fold-right (f l acc)
  (if (consp l)
      (funcall f (car l) (fold-right f (cdr l) acc))
    acc))

In [ ]:
; Example: sum using fold-left
(defun fold-sum (l)
  (fold-left (lambda (acc x) (+ acc x)) 0 l))

In [ ]:
; Example: product using fold-left
(defun fold-product (l)
  (fold-left (lambda (acc x) (* acc x)) 1 l))

In [ ]:
; Example: reverse using fold-left
(defun fold-reverse (l)
  (fold-left (lambda (acc x) (cons x acc)) nil l))

In [ ]:
; Theorem: fold-sum of a single element
(defthm fold-sum-singleton
  (equal (fold-sum (list x))
         (+ 0 x)))

In [ ]:
; Theorem: fold-sum distributes over append
(defthm fold-sum-append
  (implies (and (nat-listp l1)
                (nat-listp l2))
           (equal (fold-sum (append l1 l2))
                  (+ (fold-sum l1) (fold-sum l2)))))

In [ ]:
; Helper lemmas for nat-listp
(defthm nat-listp-append
  (implies (and (nat-listp l1)
                (nat-listp l2))
           (nat-listp (append l1 l2))))

In [ ]:
(defthm nat-listp-implies-true-listp
  (implies (nat-listp l)
           (true-listp l)))

In [ ]:
; Theorem: fold-product of a single element
(defthm fold-product-singleton
  (equal (fold-product (list x))
         (* 1 x)))

In [ ]:
; Lemma: fold-left with multiplication starting from non-1 accumulator
(defun fold-product-acc (acc l)
  (fold-left (lambda (a x) (* a x)) acc l))

In [ ]:
; Key lemma: fold-product-acc with multiplication
(defthm fold-product-acc-*
  (implies (and (acl2-numberp acc)
                (nat-listp l))
           (equal (fold-product-acc (* acc x) l)
                  (* acc (fold-product-acc x l)))))

In [ ]:
; Lemma: relationship between fold-product and fold-product-acc
(defthm fold-product-is-fold-product-acc
  (equal (fold-product l)
         (fold-product-acc 1 l)))

In [ ]:
; Lemma: fold-product-acc distributes over append
(defthm fold-product-acc-append
  (implies (and (acl2-numberp acc)
                (nat-listp l1)
                (nat-listp l2))
           (equal (fold-product-acc acc (append l1 l2))
                  (fold-product-acc (fold-product-acc acc l1) l2))))

In [ ]:
; Lemma: fold-product of cons
(defthm fold-product-cons
  (implies (and (natp x)
                (nat-listp l))
           (equal (fold-product (cons x l))
                  (* x (fold-product l)))))

In [ ]:
; Lemma: fold-product-acc identity
(defthm fold-product-acc-1
  (implies (nat-listp l)
           (equal (fold-product-acc 1 l)
                  (fold-product l))))

In [ ]:
; Lemma: associativity helper for fold-product-acc
(defthm fold-product-acc-associative
  (implies (and (acl2-numberp a)
                (acl2-numberp b)
                (nat-listp l))
           (equal (fold-product-acc (* a b) l)
                  (* a (fold-product-acc b l)))))

In [ ]:
; Lemma: fold-product returns a number
(defthm fold-product-is-number
  (implies (nat-listp l)
           (acl2-numberp (fold-product l))))

In [ ]:
; Lemma: fold-product-acc returns a number
(defthm fold-product-acc-is-number
  (implies (and (acl2-numberp acc)
                (nat-listp l))
           (acl2-numberp (fold-product-acc acc l))))

In [ ]:
; Lemma: fold-product of nil
(defthm fold-product-nil
  (equal (fold-product nil) 1))

In [ ]:
; Lemma: fold-product-acc of nil
(defthm fold-product-acc-nil
  (implies (acl2-numberp acc)
           (equal (fold-product-acc acc nil) acc)))

In [ ]:
; Lemma: multiplication with 1
(defthm *-1-left
  (implies (acl2-numberp x)
           (equal (* 1 x) x)))

In [ ]:
; Lemma: nested fold-product-acc
(defthm fold-product-acc-nested
  (implies (and (acl2-numberp acc)
                (nat-listp l1)
                (nat-listp l2))
           (equal (fold-product-acc acc (append l1 l2))
                  (fold-product-acc (fold-product-acc acc l1) l2))))

In [ ]:
; Theorem: fold-product distributes over append
; This proof requires only associativity of multiplication, not commutativity.
; We disable commutativity-of-* to prevent the rewriter from normalizing terms
; into incompatible canonical forms. With only associativity enabled, the proof
; completes cleanly via induction.
(defthm fold-product-append
  (implies (and (nat-listp l1)
                (nat-listp l2))
           (equal (fold-product (append l1 l2))
                  (* (fold-product l1) (fold-product l2))))
  :hints (("Goal" :in-theory (disable commutativity-of-*))))

In [ ]:
; Theorem: fold-reverse is equivalent to reverse
(defthm fold-reverse-is-reverse
  (equal (fold-reverse l)
         (reverse l)))

In [ ]:
; Theorem: double reverse returns original
(defthm fold-reverse-fold-reverse
  (equal (fold-reverse (fold-reverse l))
         (true-list-fix l)))

In [ ]:
; Theorem: fold-left and fold-right for addition
(defun fold-right-sum (l)
  (fold-right (lambda (x acc) (+ x acc)) l 0))

In [ ]:
(defthm fold-left-right-sum-equiv
  (implies (nat-listp l)
           (equal (fold-sum l)
                  (fold-right-sum l))))

In [ ]:
; Theorem: fold-left and fold-right for multiplication
(defun fold-right-product (l)
  (fold-right (lambda (x acc) (* x acc)) l 1))

In [ ]:
(defthm fold-left-right-product-equiv
  (implies (nat-listp l)
           (equal (fold-product l)
                  (fold-right-product l))))

In [ ]:
; map: apply function to each element
(defun map-fn (f l)
  (if (consp l)
      (cons (funcall f (car l))
            (map-fn f (cdr l)))
    nil))

In [ ]:
; filter: keep elements satisfying predicate
(defun filter (p l)
  (if (consp l)
      (if (funcall p (car l))
          (cons (car l) (filter p l))
        (filter p (cdr l)))
    nil))

In [ ]:
; Example: double all elements
(defun double-all (l)
  (map-fn (lambda (x) (* 2 x)) l))

In [ ]:
; Example: filter even numbers
(defun filter-even (l)
  (filter (lambda (x) (evenp x)) l))

In [ ]:
; Theorem: length of map
(defthm len-map-fn
  (equal (len (map-fn f l))
         (len l)))

In [ ]:
; Theorem: map distributes over append
(defthm map-fn-append
  (equal (map-fn f (append l1 l2))
         (append (map-fn f l1) (map-fn f l2))))

In [ ]:
; compose fold operations
(defun fold-sum-of-products (l-of-lists)
  (fold-sum (map-fn (lambda (l) (fold-product l)) l-of-lists)))